In [19]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm
import xgboost
import sklearn
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import class_weight

In [20]:
df_X_last = pd.read_csv("data/X_train_features.csv", index_col=0)
# df_X = pd.read_csv("data/X_train_features_new.csv", index_col=0)
X = pd.read_csv("data/X_train_features_final.csv", index_col=0).to_numpy()
Y = pd.read_csv("data/y_train.csv", index_col="id").to_numpy().squeeze()

Class Weights

In [21]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(Y), y=Y)

In [22]:
weights = dict()
for i in range(4):
    weights[i] = class_weights[i]

In [23]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [250, 270, 300]

max_features = ['sqrt']

max_depth = [3, 5, 8, None]

min_samples_split = [2, 5, 8, 15]

min_samples_leaf = [1]

bootstrap = [False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [250, 270, 300], 'max_features': ['sqrt'], 'max_depth': [3, 5, 8, None], 'min_samples_split': [2, 5, 8, 15], 'min_samples_leaf': [1], 'bootstrap': [False]}


In [24]:
rf = RandomForestClassifier(class_weight=weights)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, scoring='f1_micro', n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

In [25]:
rf_random.fit(X, Y)

C:\Users\Richard\miniconda3\envs\pytcu10\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 48 candidates, totalling 240 fits


RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(class_weight={0: 0.4221947194719472,
                                                                  1: 2.8876975169300225,
                                                                  2: 0.867876526458616,
                                                                  3: 7.525}),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [False],
                                        'max_depth': [3, 5, 8, None],
                                        'max_features': ['sqrt'],
                                        'min_samples_leaf': [1],
                                        'min_samples_split': [2, 5, 8, 15],
                                        'n_estimators': [250, 270, 300]},
                   random_state=42, scoring='f1_micro', verbose=2)

In [31]:
rf_random.best_params_

{'n_estimators': 250,
 'min_samples_split': 8,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [40]:
# best params
{'n_estimators': 250,
 'min_samples_split': 8,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

Evaluate

In [29]:
skf = StratifiedKFold(n_splits=5)
f1_base = []
f1_optim = []

for train_index, test_index in skf.split(X, Y):
    clf = xgboost.XGBClassifier()
    
    clf.fit(X[train_index], Y[train_index])
    y_pred = clf.predict(X[test_index])
    f1_base.append(f1_score(Y[test_index], y_pred, average='micro'))
    
    clf_optim = rf_random.best_estimator_
    clf_optim.fit(X[train_index], Y[train_index])
    y_pred = clf_optim.predict(X[test_index])
    f1_optim.append(f1_score(Y[test_index], y_pred, average='micro'))

In [18]:
# knn
print(np.mean(f1_base))
print(np.mean(f1_optim))

0.8516713251466277
0.84737082264174


In [63]:
# knn
print(np.mean(f1_base))
print(np.mean(f1_optim))

0.8450249725073314
0.8481520726417401


In [30]:
# knn
print(np.mean(f1_base))
print(np.mean(f1_optim))

0.8516713251466277
0.8471745555351907
